<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Sistema de obtención de información con NLTK utilizando un corpus de wikipedia


In [1]:
import json
import string
import random
import re # Regular Expressions (regex)
import urllib.request

import numpy as np

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import nltk
# Descargar el diccionario
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/poldrosky/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/poldrosky/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/poldrosky/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/poldrosky/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
stopwords_spanish = nltk.corpus.stopwords.words('spanish')

### Datos
Se consumirán los datos del artículo de wikipedia sobre DEBIAN

In [3]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/Debian_GNU/Linux')
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [4]:
# Demos un vistazo
article_text

'debian gnu/linux es un sistema operativo libre, desarrollado por miles de voluntarios de todo el mundo, que colaboran a través de internet.\nla dedicación de debian al software libre, su base de voluntarios, su naturaleza no comercial y su modelo de desarrollo abierto la distingue de otras distribuciones del sistema operativo gnu. todos estos aspectos y más se recogen en el llamado contrato social de debian. debian se caracteriza por no tener las últimas novedades en gnu/linux, pero sí tener el sistema operativo más estable posible. esto se logra por medio de paquetes y librerías antiguas pero con muchos meses de pruebas, asegurando la máxima estabilidad por cada versión que es lanzada por la comunidad de debian.\nnació en el año 1993, de la mano del proyecto debian, con la idea de crear un sistema gnu usando linux como núcleo. el proyecto debian es la organización responsable de su mantenimiento en la actualidad, y también desarrolla sistemas gnu basados en otros núcleos (debian gnu/

In [5]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 5816


### 2 - Preprocesamiento
- Remover caracteres especiales
- Quitar espacios o saltos

In [6]:
# Repaso de regex:
# https://docs.python.org/3/library/re.html

# Para practicar regex:
# https://regex101.com/

# el inicio con 'r' antes de cada string indica que se interprete como raw string
# '\n' es interpretado por Python como salto de linea
# r'\n' es interpretado por Python como el string formado por dos caracteres: 
#  backslash y n

# substituir con regex con espacio vacío:
text = re.sub(r'\[[0-9]*\]', ' ', article_text) # substituir los números entre corchetes
# (notar que los corchetes son interpretados literalmente por los backlsash)
text = re.sub(r'\s+', ' ', text) # substituir más de un caracter de espacio, salto de línea o tabulación

# probar en regex101 con los patrones anteriores:
# 'Hola [1], [], [ estoy bien   [123]. [12sss]. OK!   .'

In [7]:
# Demos un vistazo
text

'debian gnu/linux es un sistema operativo libre, desarrollado por miles de voluntarios de todo el mundo, que colaboran a través de internet. la dedicación de debian al software libre, su base de voluntarios, su naturaleza no comercial y su modelo de desarrollo abierto la distingue de otras distribuciones del sistema operativo gnu. todos estos aspectos y más se recogen en el llamado contrato social de debian. debian se caracteriza por no tener las últimas novedades en gnu/linux, pero sí tener el sistema operativo más estable posible. esto se logra por medio de paquetes y librerías antiguas pero con muchos meses de pruebas, asegurando la máxima estabilidad por cada versión que es lanzada por la comunidad de debian. nació en el año 1993, de la mano del proyecto debian, con la idea de crear un sistema gnu usando linux como núcleo. el proyecto debian es la organización responsable de su mantenimiento en la actualidad, y también desarrolla sistemas gnu basados en otros núcleos (debian gnu/hu

In [8]:
print("Cantidad de caracteres en el texto:", len(text))

Cantidad de caracteres en el texto: 5814


### 3 - Dividir el texto en sentencias y en palabras

In [9]:
corpus = nltk.sent_tokenize(text) # divide en oraciones
words = nltk.word_tokenize(text) # divide en términos

In [10]:
# Demos un vistazo
corpus[:10]

['debian gnu/linux es un sistema operativo libre, desarrollado por miles de voluntarios de todo el mundo, que colaboran a través de internet.',
 'la dedicación de debian al software libre, su base de voluntarios, su naturaleza no comercial y su modelo de desarrollo abierto la distingue de otras distribuciones del sistema operativo gnu.',
 'todos estos aspectos y más se recogen en el llamado contrato social de debian.',
 'debian se caracteriza por no tener las últimas novedades en gnu/linux, pero sí tener el sistema operativo más estable posible.',
 'esto se logra por medio de paquetes y librerías antiguas pero con muchos meses de pruebas, asegurando la máxima estabilidad por cada versión que es lanzada por la comunidad de debian.',
 'nació en el año 1993, de la mano del proyecto debian, con la idea de crear un sistema gnu usando linux como núcleo.',
 'el proyecto debian es la organización responsable de su mantenimiento en la actualidad, y también desarrolla sistemas gnu basados en otr

In [11]:
# Demos un vistazo
words[:20]

['debian',
 'gnu/linux',
 'es',
 'un',
 'sistema',
 'operativo',
 'libre',
 ',',
 'desarrollado',
 'por',
 'miles',
 'de',
 'voluntarios',
 'de',
 'todo',
 'el',
 'mundo',
 ',',
 'que',
 'colaboran']

In [12]:
print("Vocabulario:", len(words))

Vocabulario: 1046


### 4 - Funciones de ayuda para limpiar y procesar el input del usuario
- Lematizar los tokens de la oración
- Quitar símbolos de puntuación

In [13]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

# ord() nos da el código Unicode para un caracter dado
punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    # 1 - reduce el texto a mínuscula (string.lower())
    # 2 - quitar los simbolos de puntuacion (string.translate())
    # 3 - realiza la tokenización (nltk.word_tokenize)
    # 4 - realiza la lematización (nuestra función perform_lemmatization)
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus del artículo de wikipedia

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    # la entrada del usuario se usa para tokenizar y vectorizar
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words=stopwords_spanish)

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0: # si la similaridad coseno fue nula (ningún término en común)
        response = "Lo sieto, no puedo entenderte"
    else:
        response = corpus[similar_sentence_number] # obtener el documento del corpus más similar
    
    corpus.remove(user_input)
    return response

### 6 - Ensayar el sistema
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada. Sugerencias a ensayar:
- Software Libre
- Toy Story
- Por quien fue fundado?
- Distribuciones

In [15]:
# Se utilizará gradio para ensayar el bot
# Herramienta poderosa para crear interfaces rápidas para ensayar modelos
# https://gradio.app/
# import sys
# !{sys.executable} -m pip install gradio --quiet

In [16]:
import gradio as gr

def bot_response(human_text):
    print("Q:", human_text)    
    resp = generate_response(human_text.lower(), corpus)
    print("A:", resp)
    return resp

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

IMPORTANT: You are using gradio version 2.6.4, however version 3.14.0 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7861/

To create a public link, set `share=True` in `launch()`.


Q: Software Libre
A: uno de sus principales objetivos es separar en sus versiones el software libre del software no libre.
Q: Toy Story
A: debian se caracteriza por: los nombres de las versiones de debian linux son tomados de la película toy story.
Q: Por quien fue fundado?
A: el proyecto debian fue fundado en el año 1993 por ian murdock, después de haber estudiado en la universidad de purdue.
Q: Distribuciones
A: las primeras distribuciones de linux basadas en debian, corel linux y la stormix's linux de stormix, fueron comenzadas en 1999. aunque no estuvieron desarrolladas por mucho tiempo, estas distribuciones eran las primeras de muchas distribuciones basadas en debian.


KeyboardInterrupt: 

### Alumno

- Tomar un ejemplo de los bots utilizados (uno de los dos) y construir el propio.
- Sacar conclusiones de los resultados.

__IMPORTANTE__: Recuerde para la entrega del ejercicio debe quedar registrado en el colab las preguntas y las respuestas del BOT para que podamos evaluar el desempeño final.